# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [30]:
# All import statements needed for the project
import pathlib
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import re
import psycopg2
import shapely
import folium

from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, TIMESTAMP, Date, text
from geoalchemy2 import Geometry, WKTElement
from datetime import datetime, timedelta
from shapely.geometry import Point
from shapely.wkb import dumps
from pathlib import Path

In [31]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "NJflFVV2YiwlXmMlt4Y9jwTGO"
API_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
API_TREE='https://data.cityofnewyork.us/resource/uvpi-gqnh.json'

## Soql for NYC_311 and NYC_TREE Download
SOQL_BASE_QUERY_311 = f"{API_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000"
SOQL_QUERIES_311 = {f"query_{i}": f"{SOQL_BASE_QUERY_311}&$offset={(i-1)*1000000}" for i in range(1, 26)}
SOQL_QUERY_TREE=f"{API_TREE}?$where=created_at between '2015-01-01T00:00:00' and '2015-12-31T23:59:59'&$limit=1000000000"

DB_NAME = "GroupProject"
DB_USER = 'postgres'
DB_PASSWORD = ''
DB_HOST = 'localhost'
DB_URL=f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
DB_SCHEMA_FILE = "schema.sql"

# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [32]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

### Download and Clean NYC_311

In [30]:
def download_nyc_json_data(query):
    headers = {'X-App-Token': NYC_DATA_APP_TOKEN }
    response = requests.get(query, headers=headers)
    df = pd.read_json(response.text)
    return df

In [31]:
def download_311_data(queries_dict):
    for query_name, query in queries_dict.items():
        query_df = download_nyc_json_data(query)
        query_df.to_csv(f"{query_name}_data.csv", index=False)

In [76]:
def load_and_clean_311_data():
    all_data = pd.concat([pd.read_csv(f"{query_name}_data.csv", low_memory=False) for query_name in SOQL_QUERIES_311], ignore_index=True)

    data_311 = all_data[['unique_key', 'created_date', 'complaint_type', 'incident_zip', 'latitude', 'longitude']].copy()

    # Create a new column 'location' without modifying the original DataFrame
    data_311['location'] = gpd.points_from_xy(data_311['longitude'], data_311['latitude'], crs="EPSG:4326")

    # Drop latitude and longitude columns
    data_311 = data_311.drop(['latitude', 'longitude'], axis=1)

    # Rename columns
    data_311 = data_311.rename(columns={"incident_zip": "zipcode"})

    # Drop rows with missing zip codes
    data_311['zipcode'] = pd.to_numeric(data_311['zipcode'], errors='coerce')
    data_311 = data_311.dropna(subset=['zipcode'])

    # Change data types
    convert_dict = {'unique_key': int, 'created_date': 'datetime64[ns]', 'complaint_type': str, 'zipcode': int}
    data_311 = data_311.astype(convert_dict)

    # Create GeoDataFrame with 'location' as the geometry column
    geo_data_311 = gpd.GeoDataFrame(data_311, geometry='location')

    return geo_data_311

### Download and Clean NYC_tree

In [77]:
def download_and_clean_tree_data():
    # Download tree data for the year 2015
    tree_data = download_nyc_json_data(SOQL_QUERY_TREE)

    # Select relevant columns
    selected_columns = ['tree_id', 'status', 'health', 'spc_common', 'zipcode', 'latitude', 'longitude']
    tree_data = tree_data[selected_columns]

    # Convert latitude and longitude to a GeoDataFrame with a 'location' column
    tree_data['location'] = gpd.points_from_xy(tree_data['longitude'], tree_data['latitude'], crs="EPSG:4326")

    # Drop latitude and longitude columns
    tree_data = tree_data.drop(['latitude', 'longitude'], axis=1)

    # Rename columns
    tree_data = tree_data.rename(columns={"spc_common": "species"})

    return gpd.GeoDataFrame(tree_data, geometry='location')

### Load and Clean NYC_Zipcode

In [34]:
def load_and_clean_zipcodes_data(zipcode_datafile): 
    try:
        data = gpd.read_file(zipcode_datafile)
        
        # Select relevant columns and rename them
        data_nyc = data[['ZIPCODE', 'geometry']].rename(
            columns={"ZIPCODE": "zipcode", "geometry": "location"})
        
        # Create GeoDataFrame with the selected and renamed columns
        zipcode_nyc = gpd.GeoDataFrame(data_nyc, geometry='location')
        
        # Change data types and coordinate reference system
        zipcode_nyc = zipcode_nyc.to_crs(epsg=4326)
        zipcode_nyc = zipcode_nyc.astype({'zipcode': int})
        
        # Drop duplicate zipcodes, keeping the first occurrence
        zipcode_nyc = zipcode_nyc.drop_duplicates(subset=['zipcode'])
        
        return zipcode_nyc
    
    except Exception as e:
        print(f"Error: {e}. Could not read file: {zipcode_datafile}")
        return None

### Load and Clean NYC_Zillow

In [65]:
def load_and_clean_zillow_data(zillow_datafile):
    data = pd.read_csv(zillow_datafile)
    
    # Only get the data of "NY" state
    zillow_data_NY = data[data.City == "New York"]
    
    # Get DataFrame containing selected columns and all numeric columns
    numeric_columns = data.filter(regex='^\d').columns.tolist()
    zillow_NY_dataset = zillow_data_NY[["RegionName"] + numeric_columns]
    
    # Melt the DataFrame to transform the date columns into rows
    zillow_NY_dataset_melted = pd.melt(zillow_NY_dataset, id_vars=["RegionName"], var_name="recorded_date", value_name="rent")
    
    # Drop rows with missing values in the "value" column
    zillow_NY_dataset_melted = zillow_NY_dataset_melted.dropna(subset=["rent"])
    
    # Rename some columns
    zillow_NY_dataset_melted = zillow_NY_dataset_melted.rename(columns={"RegionName": "zipcode"})
    
    # Convert "date" column to date format
    zillow_NY_dataset_melted["recorded_date"] = pd.to_datetime(zillow_NY_dataset_melted["recorded_date"])
    
    return zillow_NY_dataset_melted

### Load all data

In [71]:
def load_all_data():
    data_311 = load_and_clean_311_data()
    data_tree = download_and_clean_tree_data()
    data_zipcode = load_and_clean_zipcodes_data(ZIPCODE_DATA_FILE)
    data_zillow = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    
    return (
        data_311,
        data_tree,
        data_zipcode,
        data_zillow
    )

In [78]:
data_311, data_tree, data_zipcode, data_zillow = load_all_data()

In [79]:
# Show basic info about each dataframe
data_311.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 5056549 entries, 9 to 24626307
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   unique_key      int64         
 1   created_date    datetime64[ns]
 2   complaint_type  object        
 3   zipcode         int64         
 4   location        geometry      
dtypes: datetime64[ns](1), geometry(1), int64(2), object(1)
memory usage: 231.5+ MB


In [80]:
# Show first 5 entries about each dataframe
data_311.head()

,unique_key,created_date,complaint_type,zipcode,location
9,59273608,2023-10-30 23:58:22,Noise - Residential,11233,POINT (-73.93110 40.68470)
10,59277713,2023-10-30 23:58:20,Noise - Street/Sidewalk,11229,POINT (-73.95575 40.60559)
17,59271083,2023-10-30 23:54:45,Noise - Street/Sidewalk,10025,POINT (-73.96521 40.80111)
18,59270832,2023-10-30 23:54:19,Noise - Residential,11102,POINT (-73.92473 40.76758)
20,59272404,2023-10-30 23:54:04,Noise - Vehicle,11434,POINT (-73.77976 40.69105)


In [81]:
data_tree.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 509525 entries, 0 to 509524
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   tree_id   509525 non-null  int64   
 1   status    509525 non-null  object  
 2   health    485498 non-null  object  
 3   species   485498 non-null  object  
 4   zipcode   509525 non-null  int64   
 5   location  509525 non-null  geometry
dtypes: geometry(1), int64(2), object(3)
memory usage: 23.3+ MB


In [82]:
data_tree.head()

,tree_id,status,health,species,zipcode,location
0,180683,Alive,Fair,red maple,11375,POINT (-73.84422 40.72309)
1,200540,Alive,Fair,pin oak,11357,POINT (-73.81868 40.79411)
2,204026,Alive,Good,honeylocust,11211,POINT (-73.93661 40.71758)
3,204337,Alive,Good,honeylocust,11211,POINT (-73.93446 40.71354)
4,189565,Alive,Good,American linden,11215,POINT (-73.97598 40.66678)


In [83]:
data_zipcode.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 248 entries, 0 to 262
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zipcode   248 non-null    int64   
 1   geometry  248 non-null    geometry
dtypes: geometry(1), int64(1)
memory usage: 5.8 KB


In [84]:
data_zipcode.head()

,zipcode,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [85]:
data_zillow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9039 entries, 5 to 15224
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   zipcode        9039 non-null   int64         
 1   recorded_date  9039 non-null   datetime64[ns]
 2   rent           9039 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 282.5 KB


In [86]:
data_zillow.head()

,zipcode,recorded_date,rent
5,11226,2015-01-31,1944.609891
7,10025,2015-01-31,3068.951823
13,11206,2015-01-31,2482.829299
14,11221,2015-01-31,2125.738807
20,11235,2015-01-31,1687.789898


## Part 2: Storing Data

In [33]:
## first step: create database 'GroupProject'in PostgreSQL
!createdb GroupProject

createdb: error: database creation failed: ERROR:  database "GroupProject" already exists


In [34]:
!psql --dbname GroupProject -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [54]:
def create_tables(engine):
    metadata = MetaData()

    nyc_311 = Table(
        'nyc_311', metadata,
        Column('unique_key', Integer, primary_key=True),
        Column('created_date', TIMESTAMP),
        Column('complaint_type', String),
        Column('zipcode', Integer),
        Column('location', Geometry('POINT', srid=4326))
    )

    nyc_tree = Table(
        'nyc_tree', metadata,
        Column('tree_id', Integer, primary_key=True),
        Column('status', String),
        Column('health', String),
        Column('species', String),
        Column('zipcode', Integer),
        Column('location', Geometry('POINT', srid=4326))
    )

    nyc_zipcode = Table(
        'nyc_zipcode', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('location', Geometry('POLYGON', srid=4326))
    )
    
    nyc_zillow = Table(
        'nyc_zillow', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('recorded_date', Date),
        Column('rent', Float)
    )

    metadata.create_all(engine)

In [55]:
def create_indexes(connection):
    sql_create_indexes = """
    CREATE INDEX IF NOT EXISTS idx_nyc_311_location ON "nyc_311" USING gist (location);
    CREATE INDEX IF NOT EXISTS idx_nyc_tree_location ON "nyc_tree" USING gist (location);
    CREATE INDEX IF NOT EXISTS idx_nyc_zipcode_location ON "nyc_zipcode" USING gist (location);
    """

    with connection.cursor() as cursor:
        cursor.execute(sql_create_indexes)

In [15]:
engine = create_engine(DB_URL)
create_tables(engine)

NameError: name 'create_tables' is not defined

In [57]:
with psycopg2.connect(DB_URL) as connection:
    create_indexes(connection)

### Add Data to Database

In [58]:
def write_dataframes_to_table(df_name, table_name):
    engine = create_engine(DB_URL)
    
    # if it cantains geometry data, use "gpd.to_postgis", else use "pd.to_sql"
    try:
        df_name.to_postgis(table_name, engine, if_exists='replace', index=False)
    except:
        df_name.to_sql(table_name, engine, if_exists='replace', index=False)

In [59]:
write_dataframes_to_table(data_311, 'nyc_311')
write_dataframes_to_table(data_tree, 'nyc_tree')
write_dataframes_to_table(data_zipcode,"nyc_zipcode")
write_dataframes_to_table(data_zillow,"nyc_zillow")

## Part 3: Understanding the Data

### Query 1 : Which area might be more calm to live in?
Between October 1st, 2022 and September 30th, 2023 (inclusive), find the number of 311 complaints per zip code. 

The query result should have two columns, one row per zip code, with the number of complaints in descending order.

In [20]:
# Helper function to write the queries to file
# Setting the filename for the SQL query file
def write_query_to_file(query, outfile):
    with open(outfile, 'w') as file:
        file.write(query)  # Writing the SQL query to the specified file

In [26]:
# Setting the filename for the SQL query file
QUERY_1_FILENAME = QUERY_DIR / "top_zipcodes_by_complaints.sql"


# SQL query to find the number of 311 complaints per zip code
QUERY_1 = """
SELECT zipcode, COUNT(*) AS complaint_count
FROM nyc_311
WHERE created_date BETWEEN '2022-10-01' AND '2023-09-30'
GROUP BY zipcode
ORDER BY complaint_count DESC;
"""

In [28]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_1))
    print(f"Zip Code\tComplaint Count")
    for row in result:
        print(f"{row.zipcode}｜{row.complaint_count}")

Zip Code	Complaint Count
10023｜15595
10031｜12439
10456｜12291
10025｜11903
10453｜11408
10457｜11086
10458｜10642
10032｜10619
10452｜10544
10468｜10309
11226｜10184
10466｜10067
11201｜10051
11221｜10017
11414｜9725
10002｜8889
11238｜8580
11207｜8504
10467｜8237
10033｜8201
10027｜8089
10034｜7676
11216｜7518
10011｜7290
11237｜7277
10009｜7234
11206｜6838
11368｜6710
10029｜6627
11208｜6505
10040｜6490
11225｜6374
10026｜6363
11211｜6193
11385｜6190
10472｜6178
11212｜6110
10463｜5922
10003｜5848
10460｜5705
10030｜5494
10019｜5461
10462｜5300
11233｜5239
11372｜4976
10016｜4970
11220｜4916
11366｜4886
11249｜4884
11419｜4860
11377｜4784
11213｜4757
11373｜4750
11433｜4723
10028｜4679
10024｜4556
11101｜4545
10451｜4464
10035｜4430
10455｜4402
11203｜4398
11217｜4346
10459｜4338
11231｜4297
11420｜4280
11103｜4217
11222｜4142
11375｜4011
11209｜3905
11235｜3618
10454｜3528
11215｜3525
10473｜3460
10036｜3445
11357｜3344
10014｜3340
10128｜3306
11435｜3286
11229｜3284
11223｜3269
11234｜3267
11218｜3222
10001｜3207
11230｜3201
10012｜3184
10461｜3164
11106｜3145
1122

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2: Where has the most greenery?
Using just the trees table, which 10 zip codes have the most trees?

The query result should have two columns, 10 rows. The rows should be sorted by the total number of trees, descending.

In [23]:
QUERY_2_FILENAME = QUERY_DIR / "zipcodes_by_tree_count.sq"

QUERY_2 = """
SELECT zipcode, COUNT(*) AS tree_count
FROM nyc_tree
WHERE status = 'Alive'
GROUP BY zipcode
ORDER BY tree_count DESC
LIMIT 10;
"""

In [24]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_2))
    print(f"Zip Code| Tree Count")
    for row in result:
        print(f"{row.zipcode}| {row.tree_count}")

Zip Code| Tree Count
10312| 20155
10314| 15291
10306| 11905
10309| 11133
11208| 7762
11207| 7456
11385| 7382
11230| 6936
10308| 6479
10305| 6239


In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3: Can I afford a place in the areas with the most trees?
Of the 10 zip codes with the most trees, for the month of August 2023, what is the average rent by zip code?

The query should have a JOIN statement. The query result should have two columns (not three) and 10 rows. The rows should be sorted by the total number of trees, descending. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128. 


In [ ]:
QUERY_3_FILENAME = QUERY_DIR / "average_rent_top_tree_zipcodes.sql"

QUERY_3 = """
SELECT 
    tree_counts.zipcode, 
    TO_CHAR(AVG(z.rent), 'FM9,999,999.00') as average_rent
FROM 
    (
        SELECT 
            zipcode, 
            COUNT(*) AS tree_count
        FROM 
            nyc_tree
        WHERE 
            status = 'Alive'
        GROUP BY 
            zipcode
        ORDER BY 
            COUNT(*) DESC
        LIMIT 10
    ) AS tree_counts
JOIN 
    nyc_zillow z ON tree_counts.zipcode = z.zipcode
WHERE 
    z.recorded_date BETWEEN '2023-08-01' AND '2023-08-31'
GROUP BY 
    tree_counts.zipcode, tree_counts.tree_count
ORDER BY 
    tree_counts.tree_count DESC
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_3))
    print(f"Zip Code| Average Rent")
    for row in result:
        print(f"{row.zipcode}| {row.average_rent}")

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4: Could there be a correlation between an area’s rent, the number of its trees, and the number of 311 complaints?

For the month of January 2023, return the 5 zip codes with the lowest average rent, and 5 zipcodes of the highest average rent, and include the tree count and complaint count for each zip code by using JOIN statements.

The query result should have 4 columns (zip code, average rent, tree count, and complaint count) and 10 rows: five with the highest average rent, and five with the lowest average rent. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128.

In [ ]:
QUERY_4_FILENAME = QUERY_DIR / "rent_tree_complaint_correlation.sql"

# SQL query
QUERY_4 = """
WITH RentData AS (
    SELECT 
        zipcode, 
        AVG(rent) AS average_rent
    FROM 
        nyc_zillow
    WHERE 
        recorded_date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY 
        zipcode
), RentRank AS (
    SELECT 
        zipcode, 
        average_rent,
        RANK() OVER (ORDER BY average_rent DESC) AS high_rank,
        RANK() OVER (ORDER BY average_rent ASC) AS low_rank
    FROM 
        RentData
), TreeData AS (
    SELECT 
        zipcode, 
        COUNT(*) AS tree_count
    FROM 
        nyc_tree
    GROUP BY 
        zipcode
), ComplaintData AS (
    SELECT 
        zipcode, 
        COUNT(*) AS complaint_count
    FROM 
        nyc_311
    WHERE 
        created_date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY 
        zipcode
)
SELECT 
    r.zipcode, 
    TO_CHAR(r.average_rent, 'FM9,999,999.00') as formatted_rent, 
    t.tree_count, 
    c.complaint_count
FROM 
    RentRank r
JOIN 
    TreeData t ON r.zipcode = t.zipcode
JOIN 
    ComplaintData c ON r.zipcode = c.zipcode
WHERE 
    r.high_rank <= 5 OR r.low_rank <= 5
ORDER BY 
    r.high_rank, r.low_rank
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_4))
    print("Zipcode |  Average Rent |  Tree Count |  Complaint Count")
    for row in result:
        print(f"{row.zipcode} |  {row.formatted_rent} |  {row.tree_count} |  {row.complaint_count}")


In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5: Where has the most greenery (take 2)?
Rewrite Query 2 to use both the trees table and the zipcodes table. Join both tables where the coordinate point of the tree is inside the polygon boundary of the zipcode as defined in the zipcode table.

The query should have a JOIN statement. The query results should match exactly the results of Query 2.

In [ ]:
QUERY_5_FILENAME = QUERY_DIR / "greenery_with_zipcode_spatial_join.sql"

QUERY_5 = """
SELECT z.zipcode, COUNT(*) as tree_count
FROM nyc_tree t
JOIN nyc_zipcode z ON ST_Contains(z.location, t.location)
WHERE t.status = 'Alive'
GROUP BY z.zipcode
ORDER BY tree_count DESC
LIMIT 2;
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_5))
    print("Zipcode | Tree Count")
    for row in result:
        print(f"{row.zipcode} | {row.tree_count}")

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6: What is the immediate area like?
Using the following coordinate pair on campus, which trees are within ½ mile radius of this point? Latitude: 40.80737875669467, Longitude: -73.96253174434912

The result should have 5 columns (ID, species, health, status, and coordinate location of each tree).

You may use regular Python to help construct the geometric object needed for setting the radius in the query.

In [ ]:
# Latitude and Longitude
latitude = 40.80737875669467
longitude = -73.96253174434912

# Construct point using Well-Known Text (WKT) format
point_wkt = WKTElement(f'POINT({longitude} {latitude})', srid=4326)

In [ ]:
QUERY_6_FILENAME = QUERY_DIR / "trees_near_coordinate.sql"

QUERY_6 = f"""
SELECT 
    tree_id, 
    species, 
    health, 
    status, 
    location
FROM 
    nyc_tree
WHERE 
    ST_DWithin(location::geography, '{point_wkt}'::geography, 804.672)
"""


In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_6), point=point_wkb)
    print("ID | Species | Health | Status | Location")
    for row in result:
        print(f"{row.tree_id} | {row.species} | {row.health} | {row.status} | {row.location}")

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1: What can I expect to put up with in NYC?
First, find the top 3 complaint types for October 1st, 2022 to September 30th, 2023 (inclusive). 

Then, create an appropriate visualization for the number of complaints per day over $timeframe for those complaint types.  

In [ ]:
def get_data_for_visual_1():
    # SQL query to get the number of complaints per day for the top 3 complaint types
    complaints_per_day_query = """
    SELECT 
        created_date::date, 
        complaint_type, 
        COUNT(*) as complaint_count
    FROM 
        nyc_311
    WHERE 
        created_date BETWEEN '2022-10-01' AND '2023-09-30'
        AND complaint_type IN (
            SELECT complaint_type
            FROM nyc_311
            WHERE created_date BETWEEN '2022-10-01' AND '2023-09-30'
            GROUP BY complaint_type
            ORDER BY COUNT(*) DESC
            LIMIT 3
        )
    GROUP BY created_date::date, complaint_type
    ORDER BY created_date
    """

    with engine.connect() as conn:
        complaints_data = pd.read_sql(complaints_per_day_query, conn)

    return complaints_data

In [ ]:
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))

    # Plotting the data for each complaint type
    for complaint_type in dataframe['complaint_type'].unique():
        filtered_data = dataframe[dataframe['complaint_type'] == complaint_type]
        axes.plot(filtered_data['created_date'], filtered_data['complaint_count'], label=complaint_type)

    # Labeling and styling
    axes.set_title("Top 3 NYC Complaint Types per Day")
    axes.set_xlabel("Date")
    axes.set_ylabel("Number of Complaints")
    axes.legend()

    plt.show()


In [ ]:
# Retrieve the data
complaints_data = get_data_for_visual_1()

# Plot the data
plot_visual_1(complaints_data)

### Visualization 2: What are the most common complaints in the immediate area?
Create a visualization that shows the number of complaints by complaint type for the top 10 complaints in zip code 10027 for October 1st, 2018 to September 30th, 2023 (inclusive). 

In [ ]:
def get_data_for_visual_2():
    # SQL query to get the number of complaints by complaint type for zip code 10027
    complaints_query = """
    SELECT 
        complaint_type, 
        COUNT(*) as complaint_count
    FROM 
        nyc_311
    WHERE 
        zipcode = 10027 AND
        created_date BETWEEN '2018-10-01' AND '2023-09-30'
    GROUP BY 
        complaint_type
    ORDER BY 
        complaint_count DESC
    LIMIT 10
    """

    with engine.connect() as conn:
        complaints_data = pd.read_sql(complaints_query, conn)

    return complaints_data

In [ ]:
def plot_visual_2(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    # Plotting a bar chart
    axes.bar(dataframe['complaint_type'], dataframe['complaint_count'], color='skyblue')

    # Labeling and styling
    axes.set_title("Top 10 Complaints in Zip Code 10027 (Oct 2018 - Sep 2023)")
    axes.set_xlabel("Complaint Type")
    axes.set_ylabel("Number of Complaints")
    plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability

    plt.show()

In [ ]:
# Retrieve the data
complaints_data_10027 = get_data_for_visual_2()

# Plot the data
plot_visual_2(complaints_data_10027)

### Visualization 3: Is there any correlation between rent, trees, and complaints at all?

Between January 1st, 2015 and September 30th, 2023 (inclusive), create a visualization using 2 subplots that share the x-axis where one subplot shows rent compared to the number of trees by zip code, and the other subplot shows rent compared to the number of complaints by zip code.


In [ ]:
def get_data_for_visual_3():
    # Query to get average rent by zip code
    rent_query = """
    SELECT zipcode, AVG(rent) as average_rent
    FROM nyc_zillow
    WHERE recorded_date BETWEEN '2015-01-01' AND '2023-09-30'
    GROUP BY zipcode
    """

    # Query to get tree count by zip code
    tree_query = """
    SELECT zipcode, COUNT(*) as tree_count
    FROM nyc_tree
    GROUP BY zipcode
    """

    # Query to get complaint count by zip code
    complaint_query = """
    SELECT zipcode, COUNT(*) as complaint_count
    FROM nyc_311
    WHERE created_date BETWEEN '2015-01-01' AND '2023-09-30'
    GROUP BY zipcode
    """

    # Execute queries and return dataframes
    with engine.connect() as conn:
        rent_data = pd.read_sql(rent_query, conn)
        tree_data = pd.read_sql(tree_query, conn)
        complaint_data = pd.read_sql(complaint_query, conn)

    return rent_data, tree_data, complaint_data

In [ ]:
def plot_visual_3(rent_df, tree_df, complaint_df):
    # Merging dataframes on zipcode
    merged_df = rent_df.merge(tree_df, on='zipcode').merge(complaint_df, on='zipcode')

    # Creating subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

    # Subplot 1: Rent vs Tree Count
    ax1.scatter(merged_df['average_rent'], merged_df['tree_count'], color='green')
    ax1.set_title('Average Rent vs Tree Count by Zip Code')
    ax1.set_ylabel('Tree Count')

    # Subplot 2: Rent vs Complaint Count
    ax2.scatter(merged_df['average_rent'], merged_df['complaint_count'], color='red')
    ax2.set_title('Average Rent vs Complaint Count by Zip Code')
    ax2.set_xlabel('Average Rent')
    ax2.set_ylabel('Complaint Count')

    plt.show()

In [ ]:
# Retrieve the data
rent_data, tree_data, complaint_data = get_data_for_visual_3()

# Plot the data
plot_visual_3(rent_data, tree_data, complaint_data)

### Visualization 4: If I can afford more in rent, will there be fewer issues & complaints?

Create a boxplot, where the x-axis is average rent in September 2023, separated into $1000 bins (i.e. $0-1000, $1001-2000, etc), and the y-axis is the number of 311 complaints observed in each zip code between October 1, 2022 (inclusive) to September 30, 2023 (inclusive).

In [ ]:
def get_data_for_visual_4():
    # Query to get average rent by zip code for September 2023
    rent_query = """
    SELECT zipcode, AVG(rent) as average_rent
    FROM nyc_zillow
    WHERE recorded_date BETWEEN '2023-09-01' AND '2023-09-30'
    GROUP BY zipcode
    """

    # Query to get complaint count by zip code
    complaint_query = """
    SELECT zipcode, COUNT(*) as complaint_count
    FROM nyc_311
    WHERE created_date BETWEEN '2022-10-01' AND '2023-09-30'
    GROUP BY zipcode
    """

    with engine.connect() as conn:
        rent_data = pd.read_sql(rent_query, conn)
        complaint_data = pd.read_sql(complaint_query, conn)

    # Merging dataframes on zipcode
    merged_data = rent_data.merge(complaint_data, on='zipcode')

    # Categorizing rent into bins
    rent_bins = pd.cut(merged_data['average_rent'], bins=range(0, int(merged_data['average_rent'].max()) + 1000, 1000), right=False)
    merged_data['rent_bins'] = rent_bins

    return merged_data

In [ ]:
def plot_visual_4(dataframe):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Creating a boxplot
    dataframe.boxplot(column='complaint_count', by='rent_bins', ax=ax, grid=False)

    # Labeling and styling
    ax.set_title('Number of 311 Complaints vs Average Rent')
    ax.set_xlabel('Average Rent Bins ($)')
    ax.set_ylabel('Number of 311 Complaints')
    plt.suptitle('')  # Remove default Pandas title
    plt.xticks(rotation=45)

    plt.show()

In [ ]:
afford_vs_complaint = get_data_for_visual_4()
plot_visual_4(afford_vs_complaint)

### Visualization 5: Where were the recent 311 incidents reported from in the immediate area?
Create a geospatial plot of the coordinates of reported 311 incidents that happened between January 1st, 2023 and September 30th, 2023 (inclusive) within a 1 kilometer radius of the same coordinate from Query 6 in Part 3.

In [ ]:
def get_data_for_visual_5():
    # Define the coordinate
    latitude = 40.80737875669467
    longitude = -73.96253174434912
    point = Point(longitude, latitude)

    # Convert to WKB format
    point_wkb = dumps(point, hex=True)

    # Modify the query to extract longitude and latitude
    incident_query = f"""
    SELECT ST_X(location::geometry) as longitude, ST_Y(location::geometry) as latitude
    FROM nyc_311
    WHERE 
        created_date BETWEEN '2023-01-01' AND '2023-09-30'
        AND ST_DWithin(location::geography, ST_GeomFromWKB('\\x{point_wkb}', 4326)::geography, 1000)
    """

    with engine.connect() as conn:
        incidents_data = pd.read_sql(incident_query, conn)

    return incidents_data


In [ ]:
def plot_visual_5_gpd(dataframe):
    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(dataframe, geometry=gpd.points_from_xy(dataframe['longitude'], dataframe['latitude']))

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax, markersize=5, color='red', marker='o')
    plt.title("311 Incidents Near Latitude 40.8074, Longitude -73.9625 (Jan 1, 2023 - Sep 30, 2023)")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")    
    
    # Center the plot around the given coordinate
    ax.set_xlim(longitude - 0.02, longitude + 0.02)
    ax.set_ylim(latitude - 0.02, latitude + 0.02)
    
    plt.show()

In [ ]:
# Option1: Using Geopandas
incidents_data = get_data_for_visual_5()
plot_visual_5_gpd(incidents_data)

### Visualization 6: Are areas trying to improve the amount of trees in the neighborhood?

Create a geospatial plot of two sets of data: the coordinates of trees in NYC, and the coordinates of "New Tree Request" 311 complaint types that were made from October 1st, 2018 to September 30th, 2023 (inclusive).